In [66]:
from __future__ import print_function
import re
import urllib.request
import zipfile
import lxml.etree
import itertools
import numpy as np
import tensorflow as tf
import time
import pickle
import os
import random
import sys
import h5py
import pickle
import pandas as pd
from utils import *

from keras.models import Sequential, load_model, model_from_json
from keras.layers import Dense, Activation, Dropout, LSTM, GRU
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.layers.wrappers import TimeDistributed
from keras.callbacks import LearningRateScheduler
from keras import regularizers

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
# plotly.tools.set_credentials_file(username='AbdelWahabTurkmani', api_key='zTY632QGUJbqSVFms8bQ')
# plotly.tools.set_credentials_file(username='asturkmani', api_key='bR0Ez8evmzcHY6m4XG7B')
plotly.tools.set_credentials_file(username='asturkmanics', api_key='GaUFc6cWCQVW0eCz6K75')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
time_percentage = 0.7
explained_variance = 0.9
df = pd.read_csv("data/rescuetime_data-ac-min.csv")
data_pd = Clean_DF(df)
data_pd.clean_data(time_percentage=time_percentage)
data_pd.clean_df = data_pd.clean_df.reset_index()
data_pd.get_pca(explained_variance=explained_variance)
data_pd.get_day_time()

# Saving the objects:
with open('data_pd_70.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(data_pd, f)

# # Getting back the objects:
# with open('data/data_pd_80.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
#     data_pd = pickle.load(f)
# time_percentage = 0.8
# explained_variance = 0.9

In [95]:
print("Dataset size:", data_pd.clean_df.shape,'\n')
print("Number of apps that consume", time_percentage*100, "% of all users time: ",len(data_pd.popular_apps), '\n')
print("Cleaned dataset columns:",'\n', data_pd.clean_df.columns.values, '\n')
print("Number of components that explain", explained_variance*100,"% of the data: ",data_pd.pca_data.shape[1], '\n')

Dataset size: (16704, 9) 

Number of apps that consume 70.0 % of all users time:  24 

Cleaned dataset columns: 
 ['Date' 'Time Spent (seconds)' 'Activity' 'Category' 'Productivity'
 'Activity Vector' 'Productivity Score' 'Day' 'Time'] 

Number of components that explain 90.0 % of the data:  12 



In [96]:
trace = go.Scatter(x=[i for i in range(data_pd.app_idx)], y=data_pd.app_cdf[:data_pd.app_idx], mode='markers')
data= [trace]
py.iplot(data, filename='App distribution')

In [97]:
FLAGS = tf.flags
FLAGS.look_back = 12 #lookback over a 1 hour period
FLAGS.batch_size = 10
FLAGS.inputlength = data_pd.activity_vector.shape[1]
np.random.seed(7)
dataset = data_pd.activity_vector

In [98]:
categorical_data = np.round(data_pd.activity_vector)

In [99]:
import datetime as dt
def get_time(day):
    day = day + dt.timedelta(minutes=5*84)
    all_times = [day + dt.timedelta(minutes=5*x) for x in range(204)]
    return all_times

In [100]:
days = data_pd.clean_df[['Day']].values
days = np.unique(days)
total_time = [get_time(pd.to_datetime(x)) for x in days]
len(total_time[0])
# total_time

204

In [101]:
FLAGS = tf.flags
FLAGS.look_back = 12 #lookback over a 1 hour period
FLAGS.batch_size = 10
FLAGS.inputlength = data_pd.activity_vector.shape[1]
np.random.seed(7)
dataset = data_pd.activity_vector

In [102]:
# Split text into overlapping sentences with step size 3.
def split2sequences(data, maxlen):
    print('Splitting text into sequences...')
    sequencelen = maxlen
    step = 1
    sequences = []
    response = []
    for i in range(0, len(data) - sequencelen - 1, step):
        sequences.append(data[i: i + sequencelen])
        response.append(data[i + sequencelen + 1])
    return np.array(sequences), np.array(response)

In [103]:
xN, yN = split2sequences(dataset, FLAGS.look_back)

Splitting text into sequences...


In [104]:
xN.shape

(16691, 12, 24)

In [105]:
# split into train and test sets
train_size = int(len(xN) * 0.8)
test_size = len(xN) - train_size
X_train, X_test = xN[0:train_size], xN[train_size:len(dataset)]
Y_train, Y_test = yN[0:train_size], yN[train_size:len(dataset)]

In [106]:
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

X_train shape:  (13352, 12, 24)
Y_train shape:  (13352, 24)


In [107]:
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

X_test shape:  (3339, 12, 24)
Y_test shape:  (3339, 24)


In [108]:
N_HIDDEN = 32
N_DENSE = 64
LEARNING_RATE = 0.005
DECAY = 0.001
EPOCHS = 10

In [111]:
print('Building training model...')
model = Sequential()
model.add(LSTM(N_HIDDEN, input_shape=(FLAGS.look_back, FLAGS.inputlength)))
model.add(Dropout(0.2))

model.add(Dense(N_HIDDEN, activation='sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(FLAGS.inputlength, activation='softmax'))  # Add another dense layer with the desired output size.

model.compile(loss='categorical_crossentropy', optimizer = RMSprop(lr=LEARNING_RATE, clipnorm=5))

print(model.summary()) # Convenient function to see details about the network model.

Building training model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_3 (LSTM)                    (None, 32)            7296        lstm_input_3[0][0]               
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 32)            0           lstm_3[0][0]                     
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 32)            1056        dropout_5[0][0]                  
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 32)            0           dense_5[0][0]                    
________________________________________________________________

In [ ]:
model.fit(x=X_train, y=Y_train, validation_data=(X_test, Y_test), nb_epoch=EPOCHS, batch_size=FLAGS.batch_size, verbose=1)

Train on 13352 samples, validate on 3339 samples
Epoch 1/10
 1910/13352 [===>..........................] - ETA: 74s - loss: 2.1119